# Solving the Travelling Salesman problem using Simulated Annealing

In [5]:
import numpy as np
import math
import itertools
import random
import matplotlib.pyplot as plt
import scipy.stats as stats

As our benchmark dataset we will use the famous TSPLIB 95 [Euclidean Symmetrical Instances](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/). The solver expects that the uncompressed files are available in the folder `ALL_tsp`. We start by converting `.tsp` files into a distance matrix $A$, such that $a_{i, j}$ corresponds to the distance between cities $i$ and $j$. Since we are dealing with symmetrical instances, the matrix is also symmetric.

In [6]:
def dist_arr(file_path):
    x_coord = []
    y_coord = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            

            if line.startswith("EDGE_WEIGHT_TYPE"):
                if not line.endswith("EUC_2D"):
                    print("Error Incompatible Types")
                    break

            if line[0].isdigit():
                x_coord.append(float(line.split()[1]))  
                y_coord.append(float(line.split()[2]))

    dist = np.zeros((len(x_coord), len(x_coord)))

    for i in range(len(x_coord)):
        for j in range(len(x_coord)):
            dist[i, j] = math.dist([x_coord[i], y_coord[i]], [x_coord[j], y_coord[j]])

    return dist

In [7]:
dist = dist_arr("ALL_tsp/a280.tsp")
print(dist[0:9, 0:9])

[[ 0.         20.         24.08318916 32.984845   32.984845   42.75511665
  55.71355311 63.2455532  61.18823416]
 [20.          0.         18.43908891 34.17601498 42.52058325 50.47771786
  65.60487787 72.11102551 68.        ]
 [24.08318916 18.43908891  0.         16.1245155  27.78488798 33.9411255
  49.51767361 55.31726674 50.47771786]
 [32.984845   34.17601498 16.1245155   0.         16.         18.86796226
  34.40930107 39.59797975 34.40930107]
 [32.984845   42.52058325 27.78488798 16.          0.         10.
  23.32380758 30.46309242 28.28427125]
 [42.75511665 50.47771786 33.9411255  18.86796226 10.          0.
  15.62049935 21.63330765 18.43908891]
 [55.71355311 65.60487787 49.51767361 34.40930107 23.32380758 15.62049935
   0.          8.         11.3137085 ]
 [63.2455532  72.11102551 55.31726674 39.59797975 30.46309242 21.63330765
   8.          0.          8.        ]
 [61.18823416 68.         50.47771786 34.40930107 28.28427125 18.43908891
  11.3137085   8.          0.        ]]

As a utility function, we now define `full_dist` to calculate the total length of a given route. We encode it as a vector, such that the $i$-th element is the $i$-th city we visit on our tour.

In [ ]:
def full_dist(dist, route):
    total = 0
    for i in range(len(route)):
        total += dist[route[i], route[(i+1) % len(route)]]

    return total
    

We start with a simple random search to gain some intuition about the search space. The following function will generate `lim` many random routes and identify the shortest one among them.

In [9]:
def random_routes(dist, lim):
    route = list(range(0, len(dist)))

    total = math.inf
    route_opt = None
    t = 0
    while t <= lim:
        random.shuffle(route)

        if full_dist(dist, route) < total:
            total = full_dist(dist, route)
            route_opt = route
        t += 1
      

    return total, route_opt

In [15]:
print(random_routes(dist, 10**3))

(30943.411982000478, [200, 223, 229, 249, 68, 258, 222, 181, 232, 183, 14, 165, 217, 89, 161, 221, 19, 131, 67, 116, 231, 56, 76, 269, 29, 3, 142, 79, 11, 155, 235, 98, 266, 18, 204, 1, 251, 173, 279, 230, 273, 176, 208, 145, 167, 73, 17, 215, 241, 63, 24, 257, 144, 55, 90, 53, 72, 82, 175, 51, 52, 15, 278, 210, 23, 45, 201, 26, 133, 50, 195, 156, 13, 216, 225, 150, 177, 152, 178, 108, 256, 199, 238, 202, 274, 87, 138, 264, 31, 245, 41, 0, 154, 159, 88, 237, 244, 86, 20, 106, 134, 54, 242, 207, 109, 70, 91, 92, 271, 10, 74, 248, 137, 39, 224, 21, 104, 78, 141, 187, 213, 77, 49, 239, 148, 180, 189, 196, 97, 122, 260, 125, 236, 270, 16, 259, 139, 44, 120, 130, 32, 33, 118, 162, 220, 262, 146, 37, 115, 160, 151, 71, 7, 117, 9, 253, 85, 46, 206, 168, 188, 111, 95, 47, 48, 22, 158, 143, 124, 66, 6, 61, 149, 69, 219, 123, 113, 203, 94, 276, 205, 182, 250, 101, 211, 192, 247, 254, 8, 105, 147, 186, 197, 164, 227, 36, 170, 27, 255, 93, 84, 191, 261, 218, 25, 198, 233, 75, 43, 190, 128, 12, 110

For Simulated Annealing we need to define a `step` function, which mutates our current tour. Our implementation chooses a uniformly random 2-OPT move, which are described in greater detail [here](https://jmsallan.netlify.app/blog/2-opt-moves-in-the-travelling-salesman-problem/). This mutation is known to work well in practice.

In [16]:
def step(dist, route):
    # uniformly random select two non-adjacent edges
    idx_edge_1_start = random.randrange(len(route))
    idx_edge_1_end = (idx_edge_1_start + 1) % len(route)

    idx_edge_2_start = random.randrange(len(route))
    idx_edge_2_end = (idx_edge_2_start + 1) % len(route)

    while abs(idx_edge_1_start - idx_edge_2_start) <= 1:
        idx_edge_2_start = random.randrange(len(route))
        idx_edge_2_end = (idx_edge_2_start + 1) % len(route)

    # ensure first edge is before second edge
    if idx_edge_2_start < idx_edge_1_start:
        idx_edge_1_start, idx_edge_2_start = idx_edge_2_start, idx_edge_1_start
        idx_edge_1_end, idx_edge_2_end = idx_edge_2_end, idx_edge_1_end

    # full_dist(before) - full_dist(after)
    diff = dist[route[idx_edge_1_start], route[idx_edge_1_end]] + dist[route[idx_edge_2_start], route[idx_edge_2_end]] - dist[route[idx_edge_1_start], route[idx_edge_2_start]] - dist[route[idx_edge_1_end], route[idx_edge_2_end]]

    route_new = []
    route_new += route[0 : idx_edge_1_start+1]
    route_new += list(reversed(route[idx_edge_1_end : idx_edge_2_start + 1]))
    if idx_edge_2_end != 0:
        route_new += route[idx_edge_2_end :]

    return diff, route_new

Now we have everything we need to implement the [Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing) search.

In [20]:
def sim_ann(dist, initial_route, initial_temp):
    cooling_rate = 0.0001
    final_temp = 1e-3
    it = 0

    route = initial_route
    distance = full_dist(dist, route)

    temp = initial_temp

    while temp > final_temp and it <= 1_000_000:
        diff, route_new = step(dist, route)
        
        if diff <= 0:
            if -diff/temp <= 0:
                if random.random() > math.exp(-diff/temp):
                    route = route_new
                    distance -= diff

        else:
            route = route_new
            distance -= diff
        it += 1
        temp *= 1.0 - cooling_rate

    return distance, route

As an example, we run this function with a randomised route as a starting point. The initial temperature is chosen using the common strategy $-\frac{\mu(f)}{\ln(0.8)}$, where we estimate the average length $\mu(f)$ using our random search. We can see that the route is more than ten times shorter than the one provided by `random_routes`.

In [21]:
route = list(range(0, len(dist)))
random.shuffle(route)

print(sim_ann(dist, route, 150_000.0))

(2996.8753046834063, [270, 15, 16, 17, 18, 131, 132, 133, 134, 135, 136, 137, 138, 148, 147, 140, 139, 265, 266, 267, 268, 269, 260, 261, 262, 263, 264, 204, 205, 206, 207, 209, 211, 212, 215, 214, 213, 210, 227, 228, 229, 230, 245, 238, 237, 236, 231, 232, 235, 234, 233, 226, 225, 224, 223, 222, 218, 217, 216, 219, 221, 220, 194, 195, 200, 199, 201, 202, 203, 143, 142, 141, 146, 145, 144, 198, 197, 196, 193, 192, 191, 190, 189, 187, 188, 185, 186, 184, 183, 182, 181, 180, 179, 178, 149, 177, 176, 150, 151, 155, 152, 119, 118, 156, 157, 158, 175, 159, 174, 160, 161, 162, 163, 164, 165, 166, 167, 171, 170, 169, 168, 101, 100, 99, 98, 91, 90, 102, 105, 172, 173, 106, 104, 103, 107, 89, 88, 108, 109, 110, 113, 114, 116, 117, 115, 85, 64, 66, 65, 84, 83, 86, 112, 111, 87, 82, 81, 80, 78, 79, 92, 97, 96, 95, 93, 94, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 57, 56, 55, 44, 45, 54, 53, 52, 46, 47, 51, 50, 49, 48, 36, 35, 32, 33, 34, 37, 38, 39, 40, 41, 42, 43, 58, 63, 62, 61, 60, 59, 120, 

(c) Julian Hein